In [1]:
# First things first...

import cv2
import random
import pandas as pd
import statistics

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Taken from pretrained_example.py
import os
import pickle
import PIL.Image
import numpy as np
import dnnlib
import dnnlib.tflib as tflib
import config
from encoder.generator_model import Generator

# Off-the-shelf recognizer
import face_recognition

# Plotting
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image, ImageDraw, ImageFont

In [93]:
# Plot latent vectors of shape 18x512
def generate_image(latent_vector):
    latent_vector = latent_vector.reshape((1, 18, 512))
    generator.set_dlatents(latent_vector)
    img_array = generator.generate_images()[0]
    img = PIL.Image.fromarray(img_array, 'RGB')
#     return img.resize((1024, 1024))
    return img.resize((512, 512))

In [3]:
def setup():
    tflib.init_tf()
    # Load pre-trained network.
    url = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ' # karras2019stylegan-ffhq-1024x1024.pkl
    with dnnlib.util.open_url(url, cache_dir=config.cache_dir) as f:
        _G, _D, Gs = pickle.load(f)
    generator = Generator(Gs, batch_size=1, randomize_noise=False) # -- RUNNING >1 TIMES THROWS ERROR
    fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    return [_G, _D, Gs, generator, fmt]

In [4]:
# Only run once.
[_G, _D, Gs, generator, fmt] = setup()

In [132]:
# # # # # # # #

In [110]:
subjects = {}
generated_images = os.listdir('./generated_images/')
for fname in generated_images:
    subject = fname.split('d')[0]
    lr = np.load('./latent_representations/'+fname.split('.')[0] + '.npy')
    if subject in subjects:
        subjects[subject]['generated_images'].append(fname)
        subjects[subject]['latent_representations'].append(lr)            
    else:
        subjects[subject] = {
            'generated_images': [fname],
            'latent_representations': [lr]
        }
    if len(subjects[subject]['latent_representations'])==4:
        sum_lr = np.zeros((18,512))
        for lr in subjects[subject]['latent_representations']:
            sum_lr += lr            
        subjects[subject]['latent_representations_mean'] = sum_lr/4

In [133]:
len(subjects)

192

In [134]:
ensemble_sub1 = subjects['04261']

In [135]:
ensemble_sub2 = subjects['02463']

In [136]:
centroid1 = ensemble_sub1['latent_representations_mean']
centroid2 = ensemble_sub2['latent_representations_mean']

In [137]:
centroid_img1 = generate_image(centroid1)

In [138]:
centroid_img2 = generate_image(centroid2)

In [139]:
vec1_slim = np.reshape(centroid1, [1, centroid1.shape[0] * centroid1.shape[1]])
vec2_slim = np.reshape(centroid2, [1, centroid2.shape[0] * centroid2.shape[1]])

In [153]:
enc1 = face_recognition.face_encodings(np.array(centroid_img1))[0]
enc2 = face_recognition.face_encodings(np.array(centroid_img2))[0]
# enc1 = np.random.random(128)
# enc2 = np.random.random(128)
known_encodings = [enc1, enc2]

RuntimeError: Error while calling cublasCreate(&handles[new_device_id]) in file /tmp/pip-install-bhxhd8y2/dlib/dlib/cuda/cublas_dlibapi.cpp:78. code: 1, reason: CUDA Runtime API initialization failed.

In [146]:
images = []
fr_distances = []

In [147]:
steps = 60

In [148]:
z = np.empty((steps, vec1_slim.shape[1]))   

In [149]:
for i, alpha in enumerate(np.linspace(start=1.0, stop=0.0, num=steps)):

    # Linearly interpolate.
    z[i] = (alpha) * vec1_slim + (1.0-alpha) * vec2_slim

    # Reshape interpolated vector, and get interpolated image.
    curr_vec = np.reshape(z[i], [18, 512]) # back to original shape
    curr_img = np.array(generate_image(curr_vec))

    #curr_enc = face_recognition.face_encodings(curr_img)[0]
    curr_enc = np.random.random(128)
    fr_distance = face_recognition.face_distance(known_encodings, curr_enc)

    images.append(np.array(curr_img))
    fr_distances.append(fr_distance)

In [150]:
s = images[0].shape
steps = 60
videowriter =  cv2.VideoWriter('./Interpolate_Centroids.avi', cv2.VideoWriter_fourcc(*'mp4v'), steps/10, (s[1], s[0]))
for i in range(len(images)):
    videowriter.write(images[i][...,::-1])
videowriter.release()